# CS 8770 Project 1

## Part 1

### Load libs

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
import torch.optim as optim

### Load data

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor())

dset = "MNIST"

In [ ]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor())

dset = "FashionMNIST"

In [ ]:
from torch.utils.data import DataLoader

batch_size = 128

train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Model definitions

In [ ]:
class MLP(nn.Module):
    
    # H: list of hidden layer dims
    # phi: non-linearity to use
    # n_classes: num of classes to pred
    def __init__(self, H, phi=nn.ReLU(), n_classes=10):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential()
        # create hidden layers based off input list H
        H.insert(0,28*28) # input layer
        [self.layers.append(nn.Linear(h,l)).append(phi) for h, l in zip(H,H[1:])] # hidden layers
        self.layers.append(nn.Linear(H[-1],n_classes)) # output layer

    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

In [ ]:
# Single RBF Neuron
class RBFNeuron(nn.Module):

    # mu: RBF mu vector
    # sig: RBF sigma
    def __init__(self, mu, sig):
        super(RBFNeuron, self).__init__()
        self.mu = mu.clone().detach()
        self.sig = sig.clone().detach()

    def __call__(self, x):
        #top = torch.linalg.norm(x-self.mu, dim=1)
        #return torch.exp((-0.5)*(top.pow(2) / self.sig))
        return x.mean(1)

# Layer of RBF Neurons
class RBFLayer(nn.Module):

    # nin: input dim
    # nout: output dim
    # mus: list of mean vectors for RBF neurons
    # sigs: list of sigmas for RBF neurons
    def __init__(self, nin, nout, mus, sigs):
        super(RBFLayer, self).__init__()
        self.neurons = nn.ModuleList([RBFNeuron(mus[i],sigs[i]) for i in range(nout)])

    def __call__(self, x):
        return torch.tensor([f(x).detach().numpy() for f in self.neurons], dtype=torch.float32, requires_grad=False).transpose(0,1)

# Full RBF Network
class RBFNet(nn.Module):

    # mus: list of means to use in basis functions
    # sigs: list of sigmas to use in basis functions
    # n_classes: num of classes to pred
    def __init__(self, mus, sigs, n_classes=10):
        super(RBFNet, self).__init__()
        self.K = len(mus) # number of RBFs
        self.mus = nn.Parameter(mus, requires_grad=False)
        self.sigs = nn.Parameter(sigs, requires_grad=False)
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            RBFLayer(28*28, self.K, self.mus, self.sigs),
            nn.Linear(self.K, n_classes)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, n_channels, n_classes=10):
        super(CNN, self).__init__()
        self.flatten = nn.Flatten()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=n_channels, out_channels=8,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            nn.Conv2d(in_channels=8, out_channels=64,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )
        self.mlp = nn.Sequential(
            nn.Linear(7*7*64, 64),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2),
            nn.Linear(64, n_classes)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        return self.mlp(x)

### Train (& validation / test?) loops

In [ ]:
from tqdm.notebook import tqdm # status bar

def train(model, data, loss_fn, optimizer, epochs=5):

    for epoch in range(epochs):

        epoch_loss = []

        for batch, (samples, labels) in enumerate(tqdm(data)):

            # we need to convert these into tensors
            #samples = samples.type('torch.FloatTensor')
            #labels = labels.type('torch.LongTensor')

            # forward pass
            prediction = model(samples)
            loss = loss_fn(prediction, labels)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # record loss
            epoch_loss.append(loss.item())

        # keep track of loss over our batches
        #epoch_loss = statistics.mean(epoch_loss)
        print(loss)

In [ ]:
def test(model, data, loss_fn):

    for batch, (samples, labels) in enumerate(tqdm(data)):

        # we need to convert these into tensors
        #samples = samples.type('torch.FloatTensor')
        #labels = labels.type('torch.LongTensor')

        # forward pass
        prediction = model(samples)
        loss = loss_fn(prediction, labels)

    # test loss
    print(loss)

### Fit models

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# hyperparams
learning_rate = 1e-3
momentum = 0.3
epochs = 3

### MLP

In [ ]:
mlp_model = MLP([256]*3, nn.ReLU(), n_classes=10)

optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)

In [ ]:
train(mlp_model, train_dl, loss_fn, optimizer, epochs=epochs)

In [ ]:
test(mlp_model, test_dl, loss_fn)

In [ ]:
for l in range(0,8,2):
    U,S,V = torch.svd_lowrank(mlp_model.layers[l].weight, q=32)
    mlp_model.layers[l].weight.data = U @ S.diag() @ V.T

### RBF

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(1000, init='k-means++', n_init='auto', random_state=0)
kmeans.fit(train_data.data.flatten(1))
#klabels = kmeans.predict(train_data.data.flatten(1))
clusters = kmeans.cluster_centers_.astype(float)

In [ ]:
mus = torch.from_numpy(clusters)
sigs = torch.ones(len(mus))*5e6

#mus = train_data.data.flatten(1)[:512].float() # just make mus the train data itself
#sigs = torch.ones(len(mus))*1e-3

In [ ]:
x = train_data.data.flatten(1)[:batch_size]

mu = mus[0]
sig = torch.ones(1)*5e6

RBFNeuron(mu, sig)(x)

In [ ]:
rbf_model = RBFNet(mus, sigs, n_classes=10)

optimizer = optim.Adam(rbf_model.parameters(), lr=learning_rate)

In [ ]:
train(rbf_model, train_dl, loss_fn, optimizer, epochs=epochs)

### CNN

In [ ]:
cnn_model = CNN(n_channels=1, n_classes=10)

optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

In [ ]:
train(cnn_model, train_dl, loss_fn, optimizer, epochs=epochs)

In [ ]:
test(cnn_model, test_dl, loss_fn)

### Results

In [ ]:
import seaborn as sn
import pandas as pd

In [ ]:
digit_mnist_classes = np.arange(10)

fashion_mnist_classes = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

In [ ]:
C = digit_mnist_classes if dset == "MNIST" else fashion_mnist_classes

# resub because we are loading our MNIST training data set
test_dl_2 = DataLoader(dataset=test_data, shuffle=True, batch_size=1)

model = cnn_model

confusion_mat = torch.zeros((10,10))
for sample, label in tqdm(test_dl_2):
    
    label = int(label.numpy())

    prediction = model(sample)
    # take the largest output and return integer of which it was (make a classification decision)
    prediction = int(torch.argmax(prediction).numpy())
    
    confusion_mat[label,prediction] += 1
    
df_cm = pd.DataFrame(np.asarray(confusion_mat),
                     index = [C[i] for i in np.arange(10)],
                     columns = [C[i] for i in np.arange(10)])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.show()